Linear Regression Model One Hot Encoding


In [ ]:
# Import the packages for this lab
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from google.colab import files
# Import linear regression models
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split


Data Analysis


In [ ]:
uploaded = files.upload()
movie_df_train = pd.read_csv('movies_df_train.csv')

movie_df_train

Saving movies_df_train.csv to movies_df_train (7).csv


,Unnamed: 0,Movie,genres,mpaa-ratings,theatrical-distributors,Production Budget,Domestic Gross,Release Date,creative-types,production-methods,sources,Crew Value
0,0,Toy Story,adventure,g-(us),walt-disney,30000000.0,192523233.0,"November, 1995",kids-fiction,digital-animation,original-screenplay,46.021657
1,1,Crimson Tide,action,r-(us),walt-disney,55000000.0,91387195.0,"May, 1995",contemporary-fiction,live-action,original-screenplay,46.475760
2,2,Judge Dredd,action,r-(us),walt-disney,85000000.0,34687912.0,"June, 1995",science-fiction,live-action,based-on-comic-or-graphic-novel,40.164674
3,3,The Jungle Book,adventure,pg-(us),walt-disney,175000000.0,364001123.0,"April, 2016",fantasy,animation-and-live-action,based-on-fictional-book-or-short-story,28.066436
4,4,The Lion King,adventure,g-(us),walt-disney,260000000.0,543638043.0,"July, 2019",kids-fiction,animation-and-live-action,remake,35.609616
...,...,...,...,...,...,...,...,...,...,...,...,...
2503,2503,The New Mutants,horror,pg-13-(us),sony-pictures-classics,67000000.0,23855569.0,"August, 2020",super-hero,live-action,based-on-comic-or-graphic-novel,34.361398
2504,2504,The Gentlemen,action,r-(us),miramax-dimension,22000000.0,36471796.0,"December, 2019",contemporary-fiction,live-action,original-screenplay,40.861833
2505,2505,Honest Thief,thriller-or-suspense,pg-13-(us),united-artists,30000000.0,14163574.0,"October, 2020",contemporary-fiction,live-action,original-screenplay,35.646974
2506,2506,The Last Full Measure,drama,r-(us),walt-disney,20000000.0,2949212.0,"January, 2020",dramatization,live-action,based-on-real-life-events,34.623697


In [ ]:
for i in range(movie_df_train.shape[0]):
  split_string = movie_df_train.loc[i,'Release Date'].split(', ')
  a = split_string[1]
  movie_df_train.loc[i,'Release Date'] = int(a)


In [ ]:
ticket_price = {2022:10.53, 2021: 10.17, 2020:9.18,2019:9.16,2018:9.16, 2017:8.97,2016:8.65,2015:8.43,2014:8.17,2013:8.13,2012:7.96,2011:7.93,2010:7.89,2009:7.50,2008:7.19,2007:6.88,2006:6.55,2005:6.41,2004:6.21,2003:6.03,2002:5.80,2001:5.65,2000:5.39,1999:5.06,1998:4.69,1997:4.59,1996:4.42,1995:4.35}
for i in range(movie_df_train.shape[0]):
  new_value = movie_df_train.loc[i,'Domestic Gross']/ticket_price[movie_df_train.loc[i,'Release Date']]
  new_value1 = movie_df_train.loc[i,'Production Budget']/ticket_price[movie_df_train.loc[i,'Release Date']]
  movie_df_train.loc[i,'Domestic Gross'] = new_value
  movie_df_train.loc[i,'Production Budget'] = new_value1

In [ ]:
movie_df_train = movie_df_train.drop(columns = ['Unnamed: 0',])
movie_df_train

,Movie,genres,mpaa-ratings,theatrical-distributors,Production Budget,Domestic Gross,Release Date,creative-types,production-methods,sources,Crew Value
0,Toy Story,adventure,g-(us),walt-disney,6.896552e+06,4.425821e+07,1995,kids-fiction,digital-animation,original-screenplay,46.021657
1,Crimson Tide,action,r-(us),walt-disney,1.264368e+07,2.100855e+07,1995,contemporary-fiction,live-action,original-screenplay,46.475760
2,Judge Dredd,action,r-(us),walt-disney,1.954023e+07,7.974233e+06,1995,science-fiction,live-action,based-on-comic-or-graphic-novel,40.164674
3,The Jungle Book,adventure,pg-(us),walt-disney,2.023121e+07,4.208105e+07,2016,fantasy,animation-and-live-action,based-on-fictional-book-or-short-story,28.066436
4,The Lion King,adventure,g-(us),walt-disney,2.838428e+07,5.934913e+07,2019,kids-fiction,animation-and-live-action,remake,35.609616
...,...,...,...,...,...,...,...,...,...,...,...
2503,The New Mutants,horror,pg-13-(us),sony-pictures-classics,7.298475e+06,2.598646e+06,2020,super-hero,live-action,based-on-comic-or-graphic-novel,34.361398
2504,The Gentlemen,action,r-(us),miramax-dimension,2.401747e+06,3.981637e+06,2019,contemporary-fiction,live-action,original-screenplay,40.861833
2505,Honest Thief,thriller-or-suspense,pg-13-(us),united-artists,3.267974e+06,1.542873e+06,2020,contemporary-fiction,live-action,original-screenplay,35.646974
2506,The Last Full Measure,drama,r-(us),walt-disney,2.178649e+06,3.212649e+05,2020,dramatization,live-action,based-on-real-life-events,34.623697


In [ ]:

dummies = ['Genre','MPAA Rating','Distributor','Creative Type','Production Method','Source']
#dummies = ['genres','mpaa-ratings','theatrical-distributors','creative-types','production-methods','sources']
drop_columns = ['Movie','Crew Value','Production Budget', 'Domestic Gross','Release Date']
#drop_columns = ['Movie','Crew Value','Production Budget', 'Domestic Gross','Release Date']
df_one_hot = pd.get_dummies(movie_df_train.drop(columns = drop_columns),dummies)
df_final1 = movie_df_train.copy()
df_final1.drop(columns = dummies)
df_encoded = pd.concat([df_final1,df_one_hot.set_index(df_final1.index)],axis = 1)
df_encoded

,Unnamed: 0,Movie,genres,mpaa-ratings,theatrical-distributors,Production Budget,Domestic Gross,Release Date,creative-types,production-methods,...,production-methods_live-action,production-methods_stop-motion-animation,sources_based-on-comic-or-graphic-novel,sources_based-on-factual-book-or-article,sources_based-on-fictional-book-or-short-story,sources_based-on-real-life-events,sources_based-on-tv,sources_original-screenplay,sources_remake,sources_spin-off
0,0,Toy Story,adventure,g-(us),walt-disney,6.896552e+06,4.425821e+07,1995,kids-fiction,digital-animation,...,0,0,0,0,0,0,0,1,0,0
1,1,Crimson Tide,action,r-(us),walt-disney,1.264368e+07,2.100855e+07,1995,contemporary-fiction,live-action,...,1,0,0,0,0,0,0,1,0,0
2,2,Judge Dredd,action,r-(us),walt-disney,1.954023e+07,7.974233e+06,1995,science-fiction,live-action,...,1,0,1,0,0,0,0,0,0,0
3,3,The Jungle Book,adventure,pg-(us),walt-disney,2.023121e+07,4.208105e+07,2016,fantasy,animation-and-live-action,...,0,0,0,0,1,0,0,0,0,0
4,4,The Lion King,adventure,g-(us),walt-disney,2.838428e+07,5.934913e+07,2019,kids-fiction,animation-and-live-action,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2503,2503,The New Mutants,horror,pg-13-(us),sony-pictures-classics,7.298475e+06,2.598646e+06,2020,super-hero,live-action,...,1,0,1,0,0,0,0,0,0,0
2504,2504,The Gentlemen,action,r-(us),miramax-dimension,2.401747e+06,3.981637e+06,2019,contemporary-fiction,live-action,...,1,0,0,0,0,0,0,1,0,0
2505,2505,Honest Thief,thriller-or-suspense,pg-13-(us),united-artists,3.267974e+06,1.542873e+06,2020,contemporary-fiction,live-action,...,1,0,0,0,0,0,0,1,0,0
2506,2506,The Last Full Measure,drama,r-(us),walt-disney,2.178649e+06,3.212649e+05,2020,dramatization,live-action,...,1,0,0,0,0,1,0,0,0,0


In [ ]:
print(movie_df_train.groupby('Distributor').size())
df_encoded['Animation'] = df_encoded['Production Method_animation-and-live-action'] + df_encoded['Production Method_digital-animation'] + df_encoded['Production Method_hand-animation'] + df_encoded['Production Method_stop-motion-animation']
df_encoded['factual_superhero'] = df_encoded['Creative Type_super-hero'] + df_encoded['Creative Type_factual']
df_encoded['remake_spinoff'] = df_encoded['Source_remake'] +df_encoded['Source_spin-off']
df_encoded['factual_book_real_life'] = df_encoded['Source_based-on-factual-book-or-article'] + df_encoded['Source_based-on-real-life-events']
df_encoded['PG'] = df_encoded['MPAA Rating_PG'] +df_encoded['MPAA Rating_G'] + df_encoded['MPAA Rating_PG-13']
df_encoded['R_NC17'] = df_encoded['MPAA Rating_R'] +df_encoded['MPAA Rating_NC-17']
df_encoded['Comedies'] = df_encoded['Genre_Comedy'] + df_encoded['Genre_Romantic Comedy']
df_encoded['drama_musical'] = df_encoded['Genre_Musical'] + df_encoded['Genre_Drama']

for i in range(df_encoded.shape[0]):
  if df_encoded.loc[i,'Animation'] > 1:
    df_encoded.loc[i,'Animation'] = 1
  if df_encoded.loc[i,'factual_superhero'] > 1:
    df_encoded.loc[i,'factual_superhero'] = 1
  if df_encoded.loc[i,'remake_spinoff'] > 1:
    df_encoded.loc[i,'remake_spinoff'] = 1
  if df_encoded.loc[i,'factual_book_real_life'] > 1:
    df_encoded.loc[i,'factual_book_real_life'] = 1
  if df_encoded.loc[i,'PG'] > 1:
    df_encoded.loc[i,'PG'] = 1
  if df_encoded.loc[i,'R_NC17'] > 1:
    df_encoded.loc[i,'R_NC17'] = 1
  if df_encoded.loc[i,'Comedies'] > 1:
    df_encoded.loc[i,'Comedies'] = 1
  if df_encoded.loc[i,'drama_musical'] > 1:
    df_encoded.loc[i,'drama_musical'] = 1
drop_column1 = ['Production Method_animation-and-live-action','Production Method_digital-animation','Production Method_hand-animation','Production Method_stop-motion-animation','Creative Type_super-hero',
                'Creative Type_factual','Source_remake','Source_spin-off','Source_based-on-factual-book-or-article','Source_based-on-real-life-events','MPAA Rating_PG','MPAA Rating_G','MPAA Rating_PG-13','MPAA Rating_PG','MPAA Rating_NC-17',
                'Genre_Comedy','Genre_Romantic Comedy','Genre_Musical','Genre_Drama']
df_encoded_new = df_encoded.drop(columns = drop_column1)
df_encoded_new
#Creative Type Group (factual, super-hero), ()
#Production Methods Group (Animation and Live Action, Digital Animation, Hand Animation, Stop motion animation) - 128
#Sources (Remake, Spinoff), (factual book, real life events)
#Genres (Romantic Comedy, Comedy), (Drama, Musical) - 6
#Ratings (PG,PG-13,G), (NC-17,R)
#Distributor - (Miramax,Miramax-dimension) (sony, sony classic)

KeyError: ignored

In [ ]:
df_train = df_encoded[df_encoded['Release Date'] < 2021]
df_test = df_encoded[df_encoded['Release Date'] >= 2021]
drop_for_x = ['Movie','Domestic Gross','Release Date','Genre','MPAA Rating','Distributor','Creative Type','Production Method','Source']
#drop_for_x=['Movie','Domestic Gross','Release Date','genres','mpaa-ratings','theatrical-distributors','creative-types','production-methods','sources']

X_train = df_train.drop(columns = drop_for_x)
y_train = df_train['Domestic Gross']

X_test = df_test.drop(columns = drop_for_x)
y_test = df_test['Domestic Gross']

normalize=['Production Budget','Crew Value']

X_train
#X_tX_train['Crew Value']


,Production Budget,Crew Value,Genre_Action,Genre_Adventure,Genre_Comedy,Genre_Drama,Genre_Horror,Genre_Musical,Genre_Romantic Comedy,Genre_Thriller/Suspense,...,Source_remake,Source_spin-off,Animation,factual_superhero,remake_spinoff,factual_book_real_life,PG,R_NC17,Comedies,drama_musical
0,6.896552e+06,46.021657,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,1.264368e+07,46.475759,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1.954023e+07,40.164674,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,2.023121e+07,27.981081,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
4,2.838428e+07,35.609616,0,1,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2507,2.178649e+06,34.623699,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
2508,1.013072e+06,35.035282,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2535,2.781641e+06,21.157267,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2540,7.641921e+06,44.037706,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [ ]:
def standardize_data(X_train, X_out_of_sample):

    # Initialize data frame for scaled data
    X_train_standardized = X_train.copy()
    X_out_of_sample_standardized = X_out_of_sample.copy()

    # Define scaling function
    scaler = StandardScaler()

    scaler.fit(X_train) # always fit on the x_train sample

    X_train_standardized=scaler.transform(X_train) # transform and scale
    X_out_of_sample_standardized=scaler.transform(X_out_of_sample)

    return X_train_standardized, X_out_of_sample_standardized, scaler


scaler = StandardScaler().fit(X_train[normalize])
X_std = scaler.transform(X_train[normalize])
X_std_test = scaler.transform(X_test[normalize])

X_train['Production Budget']=X_std[:,0]
X_train['Crew Value']=X_std[:,1]

X_test['Production Budget']=X_std_test[:,0]
X_test['Crew Value']=X_std_test[:,1]
X_train
#X_train_scaled, X_test_scaled, scaler = standardize_data(X_train, X_test)
#X_train_scaled

,Production Budget,Crew Value,Genre_Action,Genre_Adventure,Genre_Comedy,Genre_Drama,Genre_Horror,Genre_Musical,Genre_Romantic Comedy,Genre_Thriller/Suspense,...,Source_remake,Source_spin-off,Animation,factual_superhero,remake_spinoff,factual_book_real_life,PG,R_NC17,Comedies,drama_musical
0,-0.033880,1.438706,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,0.862736,1.516739,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1.938675,0.432233,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,2.046476,-1.661414,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
4,3.318445,-0.350515,0,1,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2507,-0.769925,-0.519937,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
2508,-0.951768,-0.449210,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2535,-0.675852,-2.834029,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2540,0.082406,1.097780,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [ ]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)
train_score = linreg.score(X_train, y_train)
test_score = linreg.score(X_test,y_test)
betas = pd.Series(linreg.coef_, index=X_train.columns)
betas = betas.append(pd.Series({"Intercept": linreg.intercept_}))
betas.idxmax()
print(train_score,test_score)
X_train

0.4851686000203187 -0.3678113391235427


<ipython-input-93-150cd327d007>:6: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  betas = betas.append(pd.Series({"Intercept": linreg.intercept_}))


,Production Budget,Crew Value,Genre_Action,Genre_Adventure,Genre_Comedy,Genre_Drama,Genre_Horror,Genre_Musical,Genre_Romantic Comedy,Genre_Thriller/Suspense,...,Source_remake,Source_spin-off,Animation,factual_superhero,remake_spinoff,factual_book_real_life,PG,R_NC17,Comedies,drama_musical
0,6.896552e+06,46.021657,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,1.264368e+07,46.475759,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1.954023e+07,40.164674,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,2.023121e+07,27.981081,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
4,2.838428e+07,35.609616,0,1,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2507,2.178649e+06,34.623699,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
2508,1.013072e+06,35.035282,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2535,2.781641e+06,21.157267,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2540,7.641921e+06,44.037706,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [ ]:
lasso = LassoCV()
lasso.fit(X_train, y_train)
lasso_score = lasso.score(X_train, y_train)

betas = pd.Series(lasso.coef_, index=X_train.columns)
betas = betas.append(pd.Series({"Intercept": lasso.intercept_}))
print(lasso_score, betas)
betas.idxmax()

0.41474010975837317 Production Budget                                1.129013e+00
Crew Value                                       0.000000e+00
Genre_Action                                    -0.000000e+00
Genre_Adventure                                  0.000000e+00
Genre_Comedy                                     0.000000e+00
Genre_Drama                                     -0.000000e+00
Genre_Horror                                     0.000000e+00
Genre_Musical                                    0.000000e+00
Genre_Romantic Comedy                            0.000000e+00
Genre_Thriller/Suspense                         -0.000000e+00
MPAA Rating_G                                    0.000000e+00
MPAA Rating_NC-17                               -0.000000e+00
MPAA Rating_PG                                   0.000000e+00
MPAA Rating_PG-13                                0.000000e+00
MPAA Rating_R                                   -0.000000e+00
Distributor_a24                                  0

<ipython-input-94-9d708dcc316b>:6: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  betas = betas.append(pd.Series({"Intercept": lasso.intercept_}))


'Intercept'

In [ ]:
ridge = RidgeCV()
ridge.fit(X_train, y_train)
ridge_score = ridge.score(X_train, y_train)
ridge_score

betas = pd.Series(ridge.coef_, index=X_train.columns)
betas = betas.append(pd.Series({"Intercept": ridge.intercept_}))
print(ridge_score, betas)
betas.idxmax()

0.48418553167387535 Production Budget                                6.094673e+06
Crew Value                                       1.571182e+06
Genre_Action                                    -1.275974e+06
Genre_Adventure                                 -1.029289e+06
Genre_Comedy                                     2.047256e+05
Genre_Drama                                     -1.010984e+06
Genre_Horror                                     1.550348e+06
Genre_Musical                                    2.014424e+06
Genre_Romantic Comedy                            1.190449e+05
Genre_Thriller/Suspense                         -5.722953e+05
MPAA Rating_G                                    3.344756e+06
MPAA Rating_NC-17                               -7.713777e+05
MPAA Rating_PG                                  -1.399544e+06
MPAA Rating_PG-13                               -1.021131e+06
MPAA Rating_R                                   -1.527033e+05
Distributor_a24                                  1

<ipython-input-34-4d835ed4bd70>:7: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  betas = betas.append(pd.Series({"Intercept": ridge.intercept_}))


'Intercept'

In [ ]:
print("Crew Value                                       11.620999e+06")
print("Production                                     9.163488e+05")



Train Score:  0.6509503408368086
Test Score:  0.6290361312172759
